<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/ken_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KenLM Framework for Language Modeling


**Install KenLM**

Download stable release and unzip: http://kheafield.com/code/kenlm.tar.gz

Need Boost >= 1.42.0 and bjam
*   Ubuntu: sudo apt-get install libboost-all-dev
*   Mac: brew install boost; brew install bjam

Run within kenlm directory:
    
*  mkdir -p build
  *  cd build
  *  cmake ..
  *  make -j 4
 
pip install https://github.com/kpu/kenlm/archive/master.zip

For more information on KenLM see: https://github.com/kpu/kenlm and http://kheafield.com/code/kenlm/

TODO: need to get some trained language models: nli_5gram.binary and sentiment.binary. How should we do that? 



In [32]:
import kenlm
import os
import re

In [33]:
path = '/home/roberta/ammi-2019-nlp/data/amazon_reviews'
os.chdir(path)


In [22]:
cat train.txt | /home/roberta/kenlm/bin/lmplz -o 3 > amazonLM.arpa

=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:860352 2:75230912512 3:141057966080
Statistics:
1 71696 D1=0.690098 D2=0.962667 D3+=1.22676
2 1239185 D1=0.712943 D2=1.05296 D3+=1.36242
3 4834597 D1=0.772513 D2=1.0869 D3+=1.33918
Memory estimate for binary LM:
type     MB
probing 113 assuming -p 1.5
probing 120 assuming -r models -p 1.5
trie     44 without quantization
trie     24 assuming -q 8 -b 8 quantization 
trie     42 assuming -a 22 array pointer compression
trie     22 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:860352 2:19826960 3:96691940
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
####################################################################################################
=== 

In [27]:
# path = '/home/roberta'
# os.chdir(path)
# kenlm/bin/lmplz amazonLM.arpa amazonLM.klm

SyntaxError: invalid syntax (<ipython-input-27-ecd3859afa75>, line 3)

In [28]:
import kenlm
model = kenlm.LanguageModel('amazonLM.arpa')
score = model.score('i like this tutu')
score

-10.064966201782227

#### The KenLM model reports negative log likelihood, not perplexity. So we'll be converting the score and report net perplexity. The following function calculate the perpelxity, get_ppl, and find all OOV words, get_oov.

#### Pereplexity is defined as follows, $$ PPL = b^{- \frac{1}{N} \sum_{i=1}^N \log_b q(x_i)} $$ All probabilities here are in log base 10 so to convert to perplexity, we do the following $$PPL = 10^{-\log(P) / N} $$ where $P$ is the total NLL, and $N$ is the word count.

In [51]:
def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    for sent in sentences:
        sent = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", sent)
        words = sent.strip().split()
        score = lm.score(sent, bos=False, eos=False)
        word_count = len(words)
        total_wc += word_count
        total_nll += score
    ppl = 10**-(total_nll/total_wc)
    return ppl


Function for loading the data

In [56]:
sentences = [('i like this tutu')]
ppl = get_ppl(model, sentences)
ppl

230.70962836087295

In [52]:
def load_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            data.append(line)
    return data

In [59]:
def get_oov(model, data):
    oov = []
    vocab = []
    for sent in data:
        sentence = sent
        words =  sentence.split()
        vocab += words
        # Find out-of-vocabulary words
        for w in words:
            if w not in model:
                    oov.append(w)
    return set(oov), set(vocab)

In [63]:
path_to_data = '/home/roberta/ammi-2019-nlp/data/amazon_reviews/train.txt'
data = load_data(path_to_data)
data[:3]

["this is a great tutu and at a really great price . it doesn ' t look cheap at all . i ' m so glad i looked on amazon and found such an affordable tutu that isn ' t made poorly . a + + \n",
 'i bought this for my 4 yr old daughter for dance class , she wore it today for the first time and the teacher thought it was adorable . i bought this to go with a light blue long sleeve leotard and was happy the colors matched up great . price was very good too since some of these go for over $ 15 . 00 dollars . \n',
 'what can i say . . . my daughters have it in orange , black , white and pink and i am thinking to buy for they the fuccia one . it is a very good way for exalt a dancer outfit : great colors , comfortable , looks great , easy to wear , durables and little girls love it . i think it is a great buy for costumer and play too . \n']

In [65]:
oov = get_oov(model, data)
# oov[0]

set()